<a href="https://colab.research.google.com/github/devShaurya/Assignment3/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive

/content/drive


In [3]:
ls

'My Drive'/  'Shared drives'/


In [4]:
cd My Drive

/content/drive/My Drive


In [5]:
ls

 00253.gslides
 00253.pptx
 005BBB.gslides
'17110145(3)_Shaurya.ipt'
'17110145(4)_Shaurya.ipt'
 17110145.csv
 17110145.gsheet
'17110145_ShauryaAgarawal(10) (1).ipt'
'17110145_ShauryaAgarawal(10).ipt'
'17110145_Shaurya Agarawal(11).ipt.dwg.ipt'
'17110145_Shaurya Agarawal(1.).ipt.dwg.ipt'
'17110145_Shaurya Agarawal(2).ipt'
'17110145_Shaurya Agarawal(4).ipt.dwg.ipt'
'17110145_Shaurya Agarawal(5).ipt'
'17110145_Shaurya Agarawal(6).ipt'
'17110145_Shaurya Agarawal(7).ipt'
'17110145_Shaurya Agarawal(8) (1).ipt.dwg.ipt'
'17110145_Shaurya Agarawal(8).ipt.dwg.ipt'
'17110145_ShauryaAgarawal(9) (1).ipt'
'17110145_ShauryaAgarawal(9).ipt'
'17110145_Shaurya Agarawal.ipt'
 17110145_Shaurya.ipt
'Advisory on Pre-registration for students.docx'
 Airtable/
 app-debug.zip
'Application for Employment_Internship-1.docx'
 Attendance.gsheet
 Ayush_jap_edited.mp4
 Ayush_jap.mp4
 b43.zip
'BBC Hackathon.gdoc'
 biobook_cells_12.png
'BRIEF PRESENTATION ON SCHOLARSHIPS & FIN ASSISTANCE - FINAL (1) (1).pptx'
'BTech F

In [6]:
cd NLP\ Embeddings

/content/drive/My Drive/NLP Embeddings


In [7]:
ls

BERT_Fine_Tuning_Sentence_Classification.ipynb        embed_test_copy.npy
BERT_Fine_Tuning_Sentence_Classification_where.ipynb  Tatti-Copy1.ipynb
bert_test.npy                                         Tatti-Copy3.ipynb
bert_train.npy                                        ytest.npy
embed_copy.npy                                        ytrain.npy


In [8]:
!pip install keras-self-attention

In [9]:
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Bidirectional,Flatten,GRU
from keras.layers import Embedding
from keras.layers import SimpleRNN
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras import backend as K
from keras_self_attention import SeqSelfAttention
np_emb_train=np.load("bert_train.npy",allow_pickle=1)
np_emb_test=np.load("bert_test.npy",allow_pickle=1)

Using TensorFlow backend.


In [0]:
def preprocess(data):
  tweets=[]
  sentiments=[]
  arr=[]
  c=0
  for i in data:
    if(i=="\n"):
      tweets.append(arr)
      arr=[]
    else:
      temp=i.split("\t")
      if(len(temp)==3):
        sentiments.append(temp[2])
      arr.append(temp[0])
  tweet=[]
  for i in tweets:
    temp=""
    for j in i:
      temp+=j
      temp+=" "
    tweet.append(temp[:-1])
  tweets=tweet 
  sent=[]
  for i in sentiments:
    temp=i[:-1]
    if(temp=="neutral"):
      sent.append(0)
    elif(temp=="positive"):
      sent.append(1)
    else:
      sent.append(2)  

  return(tweets,sent)

In [0]:
!pip install flair

     |████████████████████████████████| 194kB 2.8MB/s 
     |████████████████████████████████| 317kB 44.6MB/s 
     |████████████████████████████████| 798kB 52.8MB/s 
     |████████████████████████████████| 645kB 55.8MB/s 
     |████████████████████████████████| 778kB 58.0MB/s 
     |████████████████████████████████| 1.0MB 62.1MB/s 
     |████████████████████████████████| 860kB 46.9MB/s 
     |████████████████████████████████| 1.0MB 52.1MB/s 
     |████████████████████████████████| 3.8MB 23.0MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 348kB 60.4MB/s 
  Created wheel for segtok: filename=segtok-1.5.7-cp36-none-any.whl size=23258 sha256=150064016c69dd6d422f2d892b8c1c0eba3b453294ad8e01258819cfae9bbd94
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=0c0d25d140779aab6b61c4463d575a30bda68f4475f9f

In [0]:
from flair.embeddings import FlairEmbeddings, BertEmbeddings

# init Flair embeddings
flair_forward_embedding = FlairEmbeddings('multi-forward')
flair_backward_embedding = FlairEmbeddings('multi-backward')

# init multilingual BERT
bert_embedding = BertEmbeddings('bert-base-multilingual-cased')

In [11]:
import torch
from torch.nn.utils.rnn import pad_sequence
print(np_emb_train[0].shape)  
print(np_emb_test.shape)  
X_train = np_emb_train
X_test = np_emb_test
emb_list = []
for i in X_train:
  emb_list.append(torch.tensor(i[:31]))
X_train = pad_sequence(emb_list, batch_first=True)
#X_test = preprocessing.sequence.pad_sequences(X_test, maxlen=500)
emb_list = []
for i in X_test:
  emb_list.append(torch.tensor(i[:31]))
X_test = pad_sequence(emb_list, batch_first=True)

(18, 3072)
(1869,)


In [0]:
l=[]
m=[]
np_emb_train1=np.load("embed_copy.npy",allow_pickle=1)
np_emb_test1=np.load("embed_test_copy.npy",allow_pickle=1)

embeddings_polarity=np_emb_train1.tolist()
for i in range(len(embeddings_polarity)):
    l.append(embeddings_polarity[i][0])
    if embeddings_polarity[i][1].lower()=='positive':
        m.append(1)
    elif embeddings_polarity[i][1].lower()=='negative':
        m.append(2)
    else:
        m.append(0)
l1=[]
m1=[]
embeddings_test=np_emb_test1.tolist()
for i in range(len(embeddings_test)):
    l1.append(embeddings_test[i][0])
    if embeddings_test[i][1].lower()=='positive':
        m1.append(1)
    elif embeddings_test[i][1].lower()=='negative':
        m1.append(2)
    else:
        m1.append(0)

In [0]:
Y_train=np.load("ytrain.npy")
Y_test=np.load("ytest.npy")

In [16]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def LSTM_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(128,input_shape=(35,3072),return_sequences=1)))
    model.add(Dropout(0.2))
    model.add(SeqSelfAttention(attention_width=15,attention_activation='sigmoid'))
    model.add(Dense(3, activation='softmax'))
    model.add(GRU(20))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc',f1_m,precision_m, recall_m])
    #print(model.summary())
    filepath="lstm.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit(X_train.numpy(), Y_train, epochs=10, batch_size=256,callbacks=callbacks_list)
    return model
model=LSTM_model()


Epoch 1/10
15131/15131 [==============================] - 14s 918us/step - loss: 1.0310 - acc: 0.4406 - f1_m: 0.0720 - precision_m: 0.2476 - recall_m: 0.0443

Epoch 00001: loss improved from inf to 1.03095, saving model to lstm.hdf5
Epoch 2/10
15131/15131 [==============================] - 11s 750us/step - loss: 0.9269 - acc: 0.5372 - f1_m: 0.3894 - precision_m: 0.6213 - recall_m: 0.2880

Epoch 00002: loss improved from 1.03095 to 0.92691, saving model to lstm.hdf5
Epoch 3/10
15131/15131 [==============================] - 11s 746us/step - loss: 0.8730 - acc: 0.5756 - f1_m: 0.4673 - precision_m: 0.6564 - recall_m: 0.3653

Epoch 00003: loss improved from 0.92691 to 0.87302, saving model to lstm.hdf5
Epoch 4/10
15131/15131 [==============================] - 11s 755us/step - loss: 0.8374 - acc: 0.6027 - f1_m: 0.5258 - precision_m: 0.6622 - recall_m: 0.4376

Epoch 00004: loss improved from 0.87302 to 0.83741, saving model to lstm.hdf5
Epoch 5/10
15131/15131 [==============================]

In [18]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, Y_test, verbose=0)
accuracy, f1_score, precision

(0.5280898877999054, 0.5249822059374177, 0.5378145287864792)